In [7]:
import pandas as pd
import math
import datetime as dt

def calculo_nomina(salario_actual, años_hasta_jubilación, ipc):
    redondeado = math.floor(años_hasta_jubilación)
    salario = salario_actual
    for i in range(0, redondeado):
        salario = salario + salario * ipc.iloc[i, 1]
    return float(salario)

def calcular_monto_mensual(capital, intereses):
    n_periodos = len(intereses)
    sumatorio = sum((1 + intereses[i]) ** (i + 1) for i in range(n_periodos))
    if sumatorio == 0:
        return None
    return capital / sumatorio

def calcular_primas_jubilacion_df(df, ipc, interes1, interes2, duracion_interes1, interes_rendimiento1, interes_rendimiento2, duracion_interes_rendimiento1):
    resultados = []
    for _, row in df.iterrows():
        salario_final = calculo_nomina(row['NOMINA BRUTA 01/01/2025'], row['AÑOS HASTA JUBILACION'], ipc)
        porcentaje_renta = min((row['EDAD'] // 4) * 0.0225, 0.19)
        m1 = (salario_final * porcentaje_renta) / 12
        
        fecha_jubilacion = f"{2025 + row['AÑOS HASTA JUBILACION']}-11-20"
        date_range = pd.date_range(start=fecha_jubilacion, periods=22 * 12, freq='MS')
        date_list = date_range.strftime('%Y-%m').tolist()
        
        int1 = ((1 + (interes1 * 0.01)) ** (1 / 12)) - 1
        int2 = ((1 + (interes2 * 0.01)) ** (1 / 12)) - 1
        año1, mes1 = duracion_interes1
        
        intereses = [int1] * (año1 * 12 + mes1) + [int2] * ((22 * 12) - (año1 * 12 + mes1))
        
        rentas = []
        valor = m1
        for fecha in date_list:
            if fecha[-2:] == '01':
                valor *= 1.03
            rentas.append(valor)
        
        df_rentas = pd.DataFrame({'Fecha': date_list, 'Intereses': intereses, 'Pagos': rentas})
        
        capital_jubilacion = 0
        for i in range(len(df_rentas) - 1, -1, -1):
            renta = float(df_rentas.iloc[i, 2]) * (1 + float(df_rentas.iloc[i, 1])) ** -1
            for interes in range(i, -1, -1):
                renta = renta * (1 + float(df_rentas.iloc[interes, 1])) ** -1
            capital_jubilacion += renta
        
        fechas = pd.date_range(start='2025-01-01', end=fecha_jubilacion, freq='MS')
        fechas_formateadas = fechas.strftime('%Y-%m')
        
        int_ren1 = ((1 + (interes_rendimiento1 * 0.01)) ** (1 / 12)) - 1
        int_ren2 = ((1 + (interes_rendimiento2 * 0.01)) ** (1 / 12)) - 1
        año2, mes2 = duracion_interes_rendimiento1
        
        intereses1 = [int_ren1] * (año2 * 12 + mes2) + [int_ren2] * (len(fechas_formateadas) - (año2 * 12 + mes2))
        df1 = pd.DataFrame({'Fecha': fechas_formateadas, 'Intereses': intereses1})
        
        capital_actual = capital_jubilacion
        for i in range(len(df1) - 1, -1, -1):
            capital_actual = capital_actual * (1 + float(df1.iloc[i, 1])) ** -1
        
        monto_mensual = calcular_monto_mensual(capital_jubilacion, df1['Intereses'].tolist())
        
        resultados.append([row['ID'], capital_jubilacion, capital_actual, monto_mensual])
    
    return pd.DataFrame(resultados, columns=['ID', 'Capital Jubilación', 'Capital Actual', 'Monto Mensual'])

# Cargar los datos y calcular resultados
df_input = pd.read_csv('Datos/Limpios/datos1_limpios.csv')
df_input["AÑOS HASTA JUBILACION"] = 65 - df_input["EDAD"]
ipc_data = pd.read_excel('Datos/Originales/Dataset_1.xlsx', sheet_name='IPC')
resultados_df = calcular_primas_jubilacion_df(df_input, ipc_data, 2, 1.5, [6,9], 2.5, 2, [7,0])
print(resultados_df)


     ID  Capital Jubilación  Capital Actual  Monto Mensual
0     1       110088.177444    79175.156594     524.148208
1     2       173414.135632    82286.603714     275.152482
2     3       131567.178429    96515.259317     677.992028
3     4        95056.256339    67023.759335     419.988768
4     5       151010.975551   104389.404182     621.561066
..  ...                 ...             ...            ...
95   96       185246.742246   130616.684772     818.479015
96   97       179189.478640    92036.021115     334.191517
97   98       186128.120327    81593.572100     254.252437
98   99       251697.468788   134500.692620     511.705503
99  100       225203.109811   132868.158189     579.687722

[100 rows x 4 columns]


In [8]:
resultados_df['Capital Actual'].sum()

np.float64(10357509.465211945)

In [9]:
resultados_df['Capital Jubilación'].sum()

np.float64(18132673.30497923)

In [10]:
resultados_df['Monto Mensual'].sum()

np.float64(48950.71178645414)